Licensed under the Apache License, Version 2.0

In [1]:
!pip install geopandas geoviews h5netcdf hvplot requests xarray

import io
import geopandas as gpd
import requests
import xarray as xr

import hvplot.xarray  # noqa
import hvplot.pandas  # noqa

In [2]:
API_KEY = "AIza...(replace with your API key)"  # @param {type: "string"}
time = "2025-05-15T20:00:00Z"  # @param {type: "string"}

In [3]:
# @title Load forecast grid with xarray
response = requests.get(
    url="https://contrails.googleapis.com/v2/grids",
    # Omit `bbox` to download the whole globe by default.
    # We load a subregion to keep this notebook filesize small.
    params={"time": time, "bbox": [-40, 20, 20, 60]},
    headers={"x-goog-api-key": API_KEY},
)
response.raise_for_status()

dataset = xr.load_dataset(io.BytesIO(response.content))
dataset

<xarray.Dataset> Size: 3MB
Dimensions:                  (longitude: 241, latitude: 161, flight_level: 18,
                              time: 1)
Coordinates:
  * longitude                (longitude) float32 964B -40.0 -39.75 ... 20.0
  * latitude                 (latitude) float32 644B 20.0 20.25 ... 59.75 60.0
  * flight_level             (flight_level) int16 36B 270 280 290 ... 430 440
  * time                     (time) datetime64[ns] 8B 2025-05-15T20:00:00
    forecast_reference_time  (time) datetime64[ns] 8B 2025-05-15T19:00:00
Data variables:
    contrails                (longitude, latitude, flight_level, time) float32 3MB ...
Attributes:
    inference_pipeline_version:  {"1747339200": {"global_south_forecast": "co...
    api_version:                 contrails.api-service_20250528.02_p0
    aircraft_class:              default

In [4]:
# @title Visualize forecast grid on a specific flight level
dataset.isel(time=0).sel(flight_level=350).hvplot(
    clim=(0, 4),
    cmap='rainbow',
    kind='image',
    x='longitude',
    y='latitude',
    legend='bottom_right',
    geo=True,
    coastline=True,
    global_extent=(),
)

:Overlay
   .Image.I     :Image   [longitude,latitude]   (contrails)
   .Coastline.I :Feature   [Longitude,Latitude]

In [5]:
# @title Load forecast regions with geopandas
response = requests.get(
    url="https://contrails.googleapis.com/v2/regions",
    params={"time": time},
    headers={"x-goog-api-key": API_KEY},
)
response.raise_for_status()

geodataframe = gpd.GeoDataFrame.from_features(response.json())
geodataframe.head()

,geometry,flight_level,threshold,time,forecast_reference_time,aircraft_class
0,"MULTIPOLYGON Z (((31.41 53.09 9144, 31.91 53.8...",300,1,2025-05-15T20:00:00+00:00,2025-05-15T19:00:00+00:00,default
1,"MULTIPOLYGON Z (((-30.12 58 9144, -26.63 58.05...",300,2,2025-05-15T20:00:00+00:00,2025-05-15T19:00:00+00:00,default
2,"MULTIPOLYGON Z (((33.38 51.75 9448.8, 33.5 53....",310,1,2025-05-15T20:00:00+00:00,2025-05-15T19:00:00+00:00,default
3,"MULTIPOLYGON Z (((-30.38 57.75 9448.8, -30 58....",310,2,2025-05-15T20:00:00+00:00,2025-05-15T19:00:00+00:00,default
4,"MULTIPOLYGON Z (((-30 58.12 9448.8, -26.95 58....",310,3,2025-05-15T20:00:00+00:00,2025-05-15T19:00:00+00:00,default


In [6]:
# @title Visualize forecast regions on a specific flight level
geodataframe[geodataframe['flight_level'] == 350].hvplot(
    clim=(0, 4),
    c='threshold',
    cmap='rainbow',
    geo=True,
    coastline=True,
    global_extent=True,
)

:Overlay
   .Polygons.I  :Polygons   [Longitude,Latitude]   (threshold)
   .Coastline.I :Feature   [Longitude,Latitude]